<a href="https://colab.research.google.com/github/shuchimishra/Tensorflow_projects/blob/main/Tensorflow_Code/Intro_to_ANN/exam/Category2_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ======================================================================
#
# Basic Datasets Question
#
# Create and train a classifier for the MNIST dataset.
# Note that the test will expect it to classify 10 classes and that the
# input shape should be the native size of the MNIST dataset which is
# 28x28 monochrome. Do not resize the data. Your input layer should accept
# (28,28) as the input shape only. If you amend this, the tests will fail.
#

import tensorflow as tf
from tensorflow import keras

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

class myCallBack(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs['val_accuracy'] > 0.99):
      print("Cancelling training since accuracy reached 99%")
      self.model.stop_training=True

def solution_model():

    #Load datasets
    mnist = tf.keras.datasets.mnist
    (train_images,train_labels),(test_images,test_labels) = mnist.load_data()

    #number of classes
    n_classes = len(set(train_labels))

    #normalize image
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    #callbacks
    callback = myCallBack()
    MCP = keras.callbacks.ModelCheckpoint(filepath='bestmodel.h5',monitor='val_accuracy',
                                        mode='auto',save_best_only=True,save_weights_only=False,verbose=1)
    RLP = keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",factor=0.1,patience=5,
                                          verbose=1,mode="auto",min_lr=0.000000001)
    ES = keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=1e-4,patience=3,verbose=1)

    # model = keras.models.Sequential()
    # model.add(keras.layers.Flatten(input_shape=(28,28,1)))
    # model.add(keras.layers.Dense(64, activation='relu'))
    # model.add(keras.layers.Dropout(0.2))
    # model.add(keras.layers.Dense(n_classes, activation='softmax'))

    model = keras.models.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='relu',input_shape=(28,28,1)),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(32, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(n_classes, activation='softmax')])

    model.summary()

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics='accuracy')

    # model.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels),
    #           callbacks=[callback,MCP,RLP,ES], verbose=1)

    model.fit(
          train_images, train_labels,
          batch_size=512,
          epochs=50,
          verbose=1,
          validation_data=(test_images, test_labels),
          callbacks=[callback,MCP,RLP,ES]
      )

    model.summary()

    #Evaluate against test images
    model.evaluate(test_images, test_labels)

    # YOUR CODE HERE
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")